In [3]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread  


#### leer DBF simpe

In [15]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            dbf =  dbfread.DBF( ruta_archivo )
            indicadores = pd.DataFrame(dbf)
            #indicadores = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/OneDrive/Escritorio/INDICADORV5.DBF' mode='r' encoding='cp1252'>


In [16]:
print(indicadores.head())

  ORD_PERI  EXP_ID   OS_NOMBRE                    NOMBRE SEXO  EDAD  \
0   202201    2729  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
1   202201    3623  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
2   202201    3659  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
3   202201    3659  OSOETSYLRA       PAOLINI,CARINA INES    F    47   
4   202201    3659  OSOETSYLRA  RIVEROS GONZALEZ,TIZIANO    M     6   

           LOCAAFI    ORD_FSOL     ORD_FEC  \
0     3 DE FEBRERO  2021-08-05  2021-08-05   
1     3 DE FEBRERO  2022-01-01  2022-01-31   
2     3 DE FEBRERO  2021-12-13  2021-12-13   
3  VILLA BALLESTER  2021-09-15  2021-09-15   
4     LOMA HERMOSA  2021-09-16  2021-09-16   

                                        NOMSOL  \
0  HOSPITAL INTERZONAL GRAL DE AGUDOS SAN JOSE   
1       SISTEMA ODONTOLOGICO PRIVADO (SOP SRL)   
2                  Sanatorio Modelo de Caseros   
3                  Sanatorio Modelo de Caseros   
4                  Sanatorio Modelo de Caseros

In [4]:
#cargar primero el excel o csv
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            indicadores = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(indicadores)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/OneDrive/Escritorio/INDICADORV5.xlsx' mode='r' encoding='cp1252'>
       ORD_PERI  EXP_ID   OS_NOMBRE                    NOMBRE SEXO  EDAD  \
0        202201    2729  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
1        202201    3623  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
2        202201    3659  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
3        202201    3659  OSOETSYLRA       PAOLINI,CARINA INES    F    47   
4        202201    3659  OSOETSYLRA  RIVEROS GONZALEZ,TIZIANO    M     6   
...         ...     ...         ...                       ...  ...   ...   
61314    202308   11574  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
61315    202308   11575  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
61316    202308   11582  OSOETSYLRA     MACHADO,LUKA VALENTIN    M    15   
61317    202309   11569  OSOETSYLRA      INEXISTENTE,AFILIADO    M    46   
61318    202309   11570  OSOETSYLRA      INEXISTENTE,AFILI

### modelado

In [7]:
diccionario = {
'CONSULTA':[420301,
420132,
420125,
420134,
420126,
420127,
420130,
420121,
420117,
420129,
420110,
420104,
420145,
420105,
420102,
420109,
420128,
420108,
420101,
420103,
1,
420120,
3,
420133,
420107,
420112,
420143,
420135,
420106,
101,
420303,
420144,
420141]}

# creo la funcion para iterar el diciccionario

def aplicar_categoria(codigo):
    for categoria, codigos in diccionario.items():
        if codigo in codigos:
            return categoria
    return "practica"
            

### aplicar diccionario y crear nuevo campo practicas_consultas

In [13]:
indicadores['CONSULTAS_PRACTICAS']=indicadores['IT_COD'].apply(aplicar_categoria)
print(indicadores[['IT_COD','CONSULTAS_PRACTICAS']])
save_path = filedialog.askdirectory()
indicadores[[
"COB_ESP",
"COMPROBANT",
"DOC_ID",
"EDAD",
"EXP_ID",
"IMP_ESP",
"IT_COD",
"IT_ITOT",
"LOCAAFI",
"LOCAORI",
"NOM_NOM",
"CONSULTAS_PRACTICAS",
"NOMBRE",
"NOMENCLA",
"NOMSOL",
"ORD_FEC",
"ORD_FSOL",
"ORD_PERI",
"ORI_NOM",
"OS_NOMBRE",
"SEXO"
]].to_excel(save_path + '\indicadoresv5.xlsx', engine='openpyxl' ,index = False )
#test

       IT_COD CONSULTAS_PRACTICAS
0      990013            practica
1      990032            practica
2      990014            practica
3       20920            practica
4       20920            practica
...       ...                 ...
61314  990010            practica
61315  990010            practica
61316     100            practica
61317  990030            practica
61318  990024            practica

[61319 rows x 2 columns]
